In [211]:
def read_file(fname):
    with open(fname,'r') as f:
        a = list(tuple(map(float,i.strip().split()[1:])) for i in f if i.strip().split()[0]=='vertex') 
    return a





In [212]:
import math
import numpy as np

def slice_and_order(points,slice_height):

    final=[]
    for height in np.arange(0.2,97,slice_height):   
        result=[]
        ordered=[]

        for i in list(range(0,len(points),3)):
            sub_result=[]
            if points[i][2] < height <= points[i+2][2]:
                sub_result.append(list(points[i][n]+(height - points[i][2])/(points[i+2][2]-points[i][2])*(points[i+2][n]-points[i][n]) for n in range(2)))
            elif points[i+2][2] < height <= points[i][2]:
                sub_result.append(list(points[i+2][n]+(height - points[i+2][2])/(points[i][2]-points[i+2][2])*(points[i][n]-points[i+2][n]) for n in range(2)))

            if points[i+1][2] < height <= points[i+2][2]:
                sub_result.append(list(points[i+1][n]+(height - points[i+1][2])/(points[i+2][2]-points[i+1][2])*(points[i+2][n]-points[i+1][n]) for n in range(2)))
            elif points[i+2][2] < height <= points[i+1][2]:
                sub_result.append(list(points[i+2][n]+(height - points[i+2][2])/(points[i+1][2]-points[i+2][2])*(points[i+1][n]-points[i+2][n]) for n in range(2)))
   
            if points[i][2] < height <= points[i+1][2]:
                sub_result.append(list(points[i][n]+(height - points[i][2])/(points[i+1][2]-points[i][2])*(points[i+1][n]-points[i][n]) for n in range(2)))
            elif points[i+1][2] < height <= points[i][2]:
                sub_result.append(list(points[i+1][n]+(height - points[i+1][2])/(points[i][2]-points[i+1][2])*(points[i][n]-points[i+1][n]) for n in range(2)))

            result.append(sub_result)
        result = [x for x in result if x]
    
        ordered.append(result[0][0])
        ordered.append(result[0][1])
        result.remove(result[0])
        while result != []:
            lel=len(result)

            for i in range(0,lel):
                if ordered[-1] == result[i][0]:
                    ordered.append(result[i][1])
                    result.remove(result[i])
                    appd = True
                    break
                else:
                    appd = False

            if appd == False:
                ordered.append(result[0][0])
                ordered.append(result[0][1])
                result.remove(result[0])

        final.append(ordered)
    return final



In [217]:
import csv
def write_gcode(final,slice_height,extrude_speed,print_temp_bed,print_temp_nozzle):
    with(open('Lab33333333333.gcode', 'w')) as f:
        me = csv.writer(f)
        me.writerow(["M140 S"+str(print_temp_bed)])
        me.writerow(["M105"])
        me.writerow(["M190 S"+str(print_temp_bed)])
        me.writerow(["M104 S"+str(print_temp_nozzle)])
        me.writerow(["M105"])
        me.writerow(["M109 S"+str(print_temp_nozzle)])
        me.writerow(["M82"])
        me.writerow(["G92 E0"])
        me.writerow(["G28"])
        me.writerow(["G91"])
        me.writerow(["G1 Z2.0 F3000"])
        me.writerow(["G1 X5 Y20 Z0.3 F5000.0"])
        me.writerow(["G92 E0"])
        me.writerow(["G90"])
        me.writerow(["G1 F2700 E-5"])
        me.writerow(["M107"])
        me.writerow(["G1 F1200 E0"])
        E = 0
        height = 0.2
        for i in range(len(final)):
            me.writerow(["G1 " + "Z" + str(round(height,1))])
            me.writerow(["G1 " + "X" + str(final[i][0][0]+50) + " Y" + str(final[i][0][1]+50) + " E" + str(E)])
            for j in range(1,len(final[i])):
                E+= extrude_speed * np.sqrt(((final[i][j][0]+50)-(final[i][j-1][0]+50))**2+((final[i][j][1]+50)-(final[i][j-1][1]+50))**2)
                me.writerow(["G1 " + "X" + str(final[i][j][0]+50) + " Y" + str(final[i][j][1]+50) + " E" + str(E)])
            height += slice_height
        me.writerow(["G0 Z100"])
        me.writerow(["G28"])



In [219]:
file = 'twist_gear_vase2.stl'
slice_height = 0.2
extrude_speed = 0.03325
print_temp_bed = 60
print_temp_nozzle = 230
points = read_file(file)
final = slice_and_order(points,slice_height)
write_gcode(final,slice_height,extrude_speed,print_temp_bed,print_temp_nozzle)